In [1]:
import boto3
from botocore.exceptions import ClientError
from pprint import pprint


client = boto3.session.Session().client(service_name='ec2', region_name='us-east-1')
ec2 = boto3.session.Session().resource(service_name='ec2', region_name='us-east-1')

## all(), filter(), limit().


In [40]:
def list_all_ec2_ids():
    """This function list all instance ids
    """
#     ids = []
#     for each_inst in ec2.instances.all():
#         ids.append(each_inst.instance_id)  
#     print(ids)
         
    each_int = [each_int.instance_id for each_int in ec2.instances.all()]
    return each_int    
    
    
instances_ids = list_all_ec2_ids()   
print(instances_ids)    
    
    

['i-00e7be8a1d6c3cef5', 'i-02c205679c1a557e0', 'i-08804f2522f76b79f']


## Using filter method without resource

In [34]:
def ec2_instance_with_filter_state():
    """Get just instances that are running
    """
    filter_running = {"Name": "instance-state-name", "Values": ["running"]}
    response = ec2.instances.filter(
        Filters=[filter_running]
    )
    for each_int in response:
        for each_tags in each_int.tags:
            print(each_tags['Value'], each_int.state.get('Name'), each_int.instance_id)
    return None
    
ec2_instance_with_filter_state()


python-boto3 running i-08fd4c5f95560e57a
python-boto3 running i-00e64ff81b325efb6


## Filter based on tags

In [49]:

def get_instance_in_prod():
    """Get just instances based on env
    """
    filter_prod = {'Name': 'tag:Name','Values': ['prod']}
#     filter_i = {'Name': 'tag:ado','Values': ['ec2']}
    response = ec2.instances.filter(Filters=[filter_prod])
    for each_instance in response:
        print(each_instance.instance_id, each_instance.state.get('Name'))
    
    return None

get_instance_in_prod()
    
    

i-08fd4c5f95560e57a running
i-00e64ff81b325efb6 running


## Using waiter object

In [57]:
def stop_all_instances():
    """Using rescource to stop all instances
    """
    print("stopping all instances...!")
    ec2.instances.stop()
    print("all instances stopped!")
    
       
stop_all_instances()    

stopping all instances...!
all instances stopped!


## To provent the behaviour above let's look at waiter object in boto3

In [70]:
def stop_all_instances_waiter():
    """Using rescource to stop all instances with waiter
    """
    # Creating waiter
    waiter = client.get_waiter('instance_stopped')
    print("stopping all instances...!")
    ec2.instances.stop()
    waiter.wait(InstanceIds=instances_ids)
    print("all instances stopped!")
    
       
stop_all_instances_waiter()   

stopping all instances...!
all instances stopped!


## Using waiter object to start instances

In [ ]:
def start_all_instances_waiter():
    """Using rescource to start all instances with waiter
    """
    # Creating waiter
    waiter = client.get_waiter('instance_running')
    print("starting all instances...!")
    ec2.instances.start()
    waiter.wait(InstanceIds=instances_ids)
    print("all instances started!")
    
       
start_all_instances_waiter()


In [ ]:
def melo_driven_script():
    """This function is to perform actions like terminate, stop, start, and exit bases on user's request
    Paremeter: opt(int)
    return: None
    """
    waiter_stop = client.get_waiter('instance_stopped')
    waiter_running = client.get_waiter('instance_running')
    waiter_terminate = client.get_waiter('instance_terminated')    
    print("This script peforms the following actions on ec2 instance.")
    print("""
        1. start
        2. stop
        3. terminate
        4. Exit
    """)
    opt = int(input("Enter your option"))
    if opt == 1:
        print("Starting ec2 instances.....")
        client.start_instances(InstanceIds=instances_ids)
        waiter_running.wait(InstanceIds=instances_ids)
        print("instances started!!😊")
    elif opt == 2:
        print("stopping ec2 instance...")
        client.stop_instances(InstanceIds=instances_ids)
        waiter_stop.wait(InstanceIds=instances_ids)
        print("instances stopped!😭")
    elif opt == 3:
        print("Terminating instance...")
        client.terminate_instances(InstanceIds=instances_ids)
        waiter_terminate.wait(InstanceIds=instances_ids)
        print("Instance terminated!🙆‍♀️")
    elif opt == 4:
        print("exiting your script")
        #sys.exit()      
    else:
        print(f"{opt} is invalid. please a valid option and try once again.")
    return None           
    
    
melo_driven_script()

This script peforms the following actions on ec2 instance.

        1. start
        2. stop
        3. terminate
        4. Exit
    


### Now that we have some  instances that are terminated let's try either stop or start instances

In [45]:
try:
    instance_running = {'Name': 'instance-state-name', 'Values': ['running', 'stopped']}
    for each_inst in ec2.instances.filter(Filters=[instance_running]):
        each_inst.stop()

except ClientError as err:
    if err.response['Error']['Code'] == 'IncorrectInstanceState':
        print("Ooops, instances could not be stopped at this time")
    else:
        raise err.response['Error']['Code']

## create function to delete on untaged and unused volumes with resource object

In [44]:
def delete_volumes_with_resource():
    """Delete All Unused, Untaged volumes using resource
    This delete_volume_withRecource describe all volume ids and identified those untaged and usued and delete
    
    Args: 
        None
    Raise:
        None
    return None    
    """
    volume_status = {'Name': 'status', 'Values': ['available']}
    for each_volume in ec2.volumes.filter(Filters=[volume_status]):
        if not each_volume.tags:
            print("Deleting untaged and and not in use volumes...")
            each_volume.delete()
        print("Done")
    return None    
        
        
delete_volumes_with_resource()    

## create function to delete on untaged and unused volumes with client object

In [54]:
def delete_untaged_volumes():
    """This volume delete untaged and unused ebs volumes using client
    """
    client = boto3.session.Session(profile_name="shared").client(service_name='ec2', region_name='us-east-1')
    
    for each_vlume in client.describe_volumes().get('Volumes'):
        if each_vlume.get('State') != 'in-use' and not 'Tags' in each_vlume:
            print("Deleting untaged and un used volumes")
            client.delete_volume(VolumeId= each_vlume['VolumeId'])

    print("Deleted untaged and un used volumes")    
    return None        


## Now we have all the volume id whose state are not in-use and not taged

In [55]:
delete_untaged_volumes()    

Deleting untaged and un used volumes
Deleting untaged and un used volumes
Deleted untaged and un used volumes
